# Overcoming the Limitations of Threads

## The GIL

In the last mission, we learned about I/O bounds, threads, and locks. We went through a few exercises where we turned code into its threaded equivalent to see if we gained any speed. Unfortunately, even when we ran multiple threads, we didn't see the performance gains we intuitively expected. For example, **`2` threads didn't make the program twice as fast**.<br>

There are two main reasons for this:
* The [Global Interpreter Lock](https://wiki.python.org/moin/GlobalInterpreterLock), or [GIL](https://en.wikipedia.org/wiki/CPython), in CPython.
* The number of cores in the underlying processor.

In the last mission, we learned about the idea of thread safety -- how if two threads write to the same resource at the same time, they can cause conflicts. We saw an example of this when multiple threads wrote to the system standard output at the same time. This caused issues with output appearing out of order, or newlines not showing up after a string.<br>

[CPython](https://en.wikipedia.org/wiki/CPython), the most commonly used [Python interpreter](http://docs.python-guide.org/en/latest/starting/which-python/), does not do memory management in a thread safe way. This means that CPython needs to prevent multiple threads from executing Python code at the same time. Although you can initialize multiple threads and execute code using them, the GIL only allows one of those threads at a time to execute Python code, via a locking mechanism.<br>

This makes threads perform poorly when parallelizing CPU-bound tasks, since only one thread can execute at a time. However, the GIL is released in certain situations, including when doing I/O operations such as waiting for network activity or reading a file. This makes threads more performant in I/O bound situations.<br>

We can demonstrate this by reading a file with two different threads, and seeing how long it takes. If they take less than twice the time it took to run the code with a single thread, then you know you're seeing some benefit. The larger the file, the greater the benefit you'll see from multiple threads, since the GIL will be released for longer, as the file data is transferred.



* Write a function that opens and reads the file `Emails.csv`.
* Run the function `100` times and time it each time.
  * Assign the times to the list `times`.
* Create two threads that both call the function and run them `100` times.
  * Assign the `times` to the list `threaded_times`.
* Find the median of `times` and `threaded_times`.
* What do the results tell you? Were they what you expected?

In [2]:
import threading
import time
import statistics

In [17]:
def open_emails():
    emails = []
    with open('../data/Emails.csv') as f:
        data = f.read()
    return data

In [18]:
times = []

for i in range(100):
    start = time.time()
    open_emails()
    times.append(time.time() - start)
    
print(statistics.median(times))

0.043221473693847656


In [19]:
threaded_times = []

for i in range(100):
    
    start = time.time()
    
    thread1 = threading.Thread(target=open_emails)
    thread2 = threading.Thread(target=open_emails)
    thread1.start()
    thread2.start()
    thread1.join()
    thread2.join()
    
    threaded_times.append(time.time() - start)
    
print(statistics.median(threaded_times))

0.08028936386108398


## Python Interpreters

Before we dive more into the GIL and how it works, let's take a minute to discuss Python interpreters. A good way to think about interpreters is by thinking about how your brain interprets the words you're seeing right now. If you didn't know English, these letters would just be drawings on a page. If you look at letters in a language you don't know, they won't appear to have any meaning:

![alphabet-interpret](https://s3.amazonaws.com/dq-content/169/esperanto.gif)

The above is the alphabet of an obscure language called [Esperanto](https://en.wikipedia.org/wiki/Esperanto). If you understand English, some of the letters look familiar, but many have strange symbols above them, or don't "click" in your mind.<br>

A Python program is fundamentally just a series of random symbols written in a specific way. Here's an example:

```python
def loop():
    for i in range(5):
        print(i)
```

If you understand Python code already, the above example is easy to read. If you don't, the example might look foreign or intimidating. Just like you need lessons before you can understand Esperanto, a computer needs to be told how to parse the syntax of the Python language and execute it. Without an interpreter, there's no way for a computer to execute your Python code.<br>

Computers don't natively understand Python. When your program "runs", what actually happens is that your Python program is translated into code the machine can run. The code is then executed, and the response is translated back into a value you can understand.<br>

The CPython interpreter takes care of translating your code into Python bytecode. The bytecode is then executed by the interpreter in a virtual machine that converts bytecode to machine code, which we'll cover more in a bit. Here's a diagram:

![cpython-interpret-diagram](https://s3.amazonaws.com/dq-content/168/bytecode.svg)

Translating to bytecode first makes it easier for the CPython interpreter to associate each command with the machine code that it needs to run. The components that translate Python code to bytecode and bytecode to machine code are written in a low-level language called **[C](https://en.wikipedia.org/wiki/C)**.

There's a small set of potential bytecode operations, and each one is mapped to machine code ahead of time. This makes running programs as fast as possible. We can use the [dis](https://docs.python.org/3/library/dis.html) package to look at the bytecode generated for our programs. The [dis.dis()](https://docs.python.org/3/library/dis.html#dis.dis) function will dissassemble a Python object and turn it into bytecode.<br>

Here's what our example function from above would look like in bytecode:

```
4           0 SETUP_LOOP              30 (to 33)
              3 LOAD_GLOBAL              0 (range)
              6 LOAD_CONST               1 (5)
              9 CALL_FUNCTION            1 (1 positional, 0 keyword pair)
             12 GET_ITER
        >>   13 FOR_ITER                16 (to 32)
             16 STORE_FAST               0 (i)

  5          19 LOAD_GLOBAL              1 (print)
             22 LOAD_FAST                0 (i)
             25 CALL_FUNCTION            1 (1 positional, 0 keyword pair)
             28 POP_TOP
             29 JUMP_ABSOLUTE           13
        >>   32 POP_BLOCK
        >>   33 LOAD_CONST               0 (None)
             36 RETURN_VALUE
```

We won't dive into how to read bytecode [here](http://akaptur.com/blog/2013/11/15/introduction-to-the-python-interpreter/), but you can find a good guide here. After the bytecode is generated, the Python interpreter loops through the bytecode, and runs a pre-generated snippet of code that corresponds to each bytecode instruction. You can [read more here](http://www.devshed.com/c/a/python/how-python-runs-programs/), or see the source code that matches the bytecode to [machine code here](https://github.com/python/cpython/blob/51ba5b7d0c194b0c2b1e5d647e70e3538b8dde3e/Python/ceval.c#L1357). It's not usually important to dive deeply into how CPython interprets code. The important things to understand are:<br>
* Python enables us to write at a high abstraction layer. This means that our code can be extremely terse, but still achieve a lot. Imagine having to write the raw bytecode above every time you wanted to write a simple loop!
* Python builds on a lot of other tools, and sometimes the performance of Python can be impacted by those tools. In this case, CPython does a lot of work for us, and is a huge help, but it also introduces some limitations, like the GIL.

Although CPython is widely used becuase it gets updates the fastest, and is the "official" interpreter, there are other Python interpreters written in other languages, such as:<br>
* [Jython](http://www.jython.org/) -- A Python interpreter that runs in the [JVM](https://en.wikipedia.org/wiki/Java_virtual_machine).
* [PyPy](https://pypy.org/) -- a faster Python interpreter.
* [IronPython](http://ironpython.net/) -- Python running on the [.NET](https://en.wikipedia.org/wiki/.NET_Framework) framework.

Each interpreter has its own tradeoffs, but it's usually best to just go with CPython. Before we move on, let's take a quick look at some bytecode using dis.


## Clinton Emails

